In [1]:
# 准备数据
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

train_labels = []
train_samples = []

# 生成数据
for i in range(50):
    # 大约5%的年轻人确实经历过副作用
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(1)

    # 大约5%的老年人没有经历过副作用
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(0)

for i in range(1000):
    # 大约95%的年轻人没有经历过副作用
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(0)

    # 大约95%的老年人确实经历过副作用
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(1)

train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
train_labels,train_samples = shuffle(train_labels,train_samples)
print(train_samples.shape)

# 通过将每个特征缩放到给定的范围来转换特征。
scaler = MinMaxScaler(feature_range=(0,1))

scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))
# 我们只是根据fit_transform（）函数默认情况下不接受一维数据的情况，将数据重塑为2D。
print(train_samples.reshape(-1,1).shape)

(2100,)
(2100, 1)


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [7]:
model.fit(x = scaled_train_samples, y = train_labels, validation_split = 0.1, batch_size = 10, epochs = 30, verbose = 2)

Epoch 1/30
189/189 - 1s - loss: 0.2621 - accuracy: 0.9349 - val_loss: 0.2498 - val_accuracy: 0.9333
Epoch 2/30
189/189 - 1s - loss: 0.2609 - accuracy: 0.9333 - val_loss: 0.2488 - val_accuracy: 0.9333
Epoch 3/30
189/189 - 1s - loss: 0.2599 - accuracy: 0.9339 - val_loss: 0.2468 - val_accuracy: 0.9333
Epoch 4/30
189/189 - 1s - loss: 0.2586 - accuracy: 0.9365 - val_loss: 0.2472 - val_accuracy: 0.9476
Epoch 5/30
189/189 - 1s - loss: 0.2577 - accuracy: 0.9349 - val_loss: 0.2441 - val_accuracy: 0.9333
Epoch 6/30
189/189 - 1s - loss: 0.2566 - accuracy: 0.9333 - val_loss: 0.2452 - val_accuracy: 0.9476
Epoch 7/30
189/189 - 1s - loss: 0.2557 - accuracy: 0.9407 - val_loss: 0.2422 - val_accuracy: 0.9333
Epoch 8/30
189/189 - 1s - loss: 0.2549 - accuracy: 0.9370 - val_loss: 0.2427 - val_accuracy: 0.9333
Epoch 9/30
189/189 - 1s - loss: 0.2540 - accuracy: 0.9370 - val_loss: 0.2409 - val_accuracy: 0.9333
Epoch 10/30
189/189 - 1s - loss: 0.2534 - accuracy: 0.9381 - val_loss: 0.2414 - val_accuracy: 0.9333

In [14]:
model.save('models/medical_trial_model.h5')

In [16]:
from tensorflow.keras.models import load_model
new_model = load_model('models/medical_trial_model.h5')

In [17]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [19]:
json_string = model.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 1], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_input"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 1], "dtype": "float32", "units": 16, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 32, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_r

In [20]:
from tensorflow.keras.models import model_from_json
model_architecture = model_from_json(json_string)

In [21]:
model_architecture.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [22]:
yaml_string = model.to_yaml()
yaml_string

'backend: tensorflow\nclass_name: Sequential\nconfig:\n  layers:\n  - class_name: InputLayer\n    config:\n      batch_input_shape: !!python/tuple\n      - null\n      - 1\n      dtype: float32\n      name: dense_input\n      ragged: false\n      sparse: false\n  - class_name: Dense\n    config:\n      activation: relu\n      activity_regularizer: null\n      batch_input_shape: !!python/tuple\n      - null\n      - 1\n      bias_constraint: null\n      bias_initializer:\n        class_name: Zeros\n        config: {}\n      bias_regularizer: null\n      dtype: float32\n      kernel_constraint: null\n      kernel_initializer:\n        class_name: GlorotUniform\n        config:\n          seed: null\n      kernel_regularizer: null\n      name: dense\n      trainable: true\n      units: 16\n      use_bias: true\n  - class_name: Dense\n    config:\n      activation: relu\n      activity_regularizer: null\n      bias_constraint: null\n      bias_initializer:\n        class_name: Zeros\n     

In [23]:
from tensorflow.keras.models import model_from_yaml
model_architecture = model_from_yaml(yaml_string)

In [24]:
model_architecture.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.save_weights('models/my_model_weights.h5')

In [29]:
model2 = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

In [30]:
model2.load_weights('models/my_model_weights.h5')